In [1]:
# importing packages
import numpy as np
import pandas as pd
import datetime as dt
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv

In [2]:
# reading in app list
app_path = 'C:/Users/Jonah Melhado/Documents/Important Docs/BrainStation/Notebooks/proj_books/Capstone stuff/data/steam.csv'
apps = pd.read_csv(app_path)

In [3]:
# Checking and creating list
app_id = apps['appid']
app_id.count()

27075

In [4]:
# Creating batches of app id's to split up scraping
app_id_0 = app_id[:5000]
app_id_1 = app_id[5000:10000]
app_id_2 = app_id[10000:15000]
app_id_3 = app_id[15000:20000]
app_id_4 = app_id[20000:27075]

In [27]:
# Below are functions used to scroll through the infinite scrolling review pages and scrape relevant info

# getting flat list from soup output
def get_list_content(data):
    """
    data of form: [name, review, hours]
    """
    x = []
    
    for i in data:
        x.append(i.contents[0])
            
    return np.array(x)

# getting flattened list of names as the format slightly differs
def get_names(data):
    x = []
    result = []
    
    for i in data:
        x.append(i.contents)
    
    for i in x:
        if len(i)==0:
            continue
        else:
            try:
                result.append(i[1].contents[0])
            except:
                try:
                    result.append(i[0].contents[0])
                except:
                    result.append(None)
            
    return np.array(result)

browser = webdriver.Chrome()

def get_data(x):
    
    for i in x:
        
        # print statement to monitor progress
        print(f"\rCurrent App: {i}", end="")

        # navigate to review page
        browser.get(f'https://steamcommunity.com/app/{i}/reviews/?browsefilter=toprated&snr=1_5_100010_')

        # if there is an age gate, click continue to bypass and access reviews
        try:
            button = browser.find_element_by_id('age_gate_btn_continue')
            button.click()
        except:
            pass

        # scroll to the bottom and establish first stopping point
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        
        # boolean for while loop
        match=False

        while(match==False):
            
            # create variable to save page checkpoint
            end = lenOfPage
            
            # give page 1 second to load after scrolling
            time.sleep(1)
            
            # scrolling
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            
            # checking place on page to see if entire page has loaded
            if end==lenOfPage:
                match=True
        
        # give end of page time to load so entire page source is acquired
        time.sleep(2)
        
        # getting page data
        source_data = browser.page_source
        
        # creating soup object
        soup = BeautifulSoup(source_data)
        
        # retrieving salient info and putting into desired format
        name = get_names(soup.find_all('div', class_="apphub_CardContentAuthorName"))
        review = get_list_content(soup.find_all('div', class_="title"))
        hours = get_list_content(soup.find_all('div', class_="hours"))
        
        # creating dataframe
        temp = pd.DataFrame([name, review, hours]).T
        temp.columns=['username', 'review', 'hours']
        
        # write to csv file 
        path = 'C:/Users/Jonah Melhado/Documents/Important Docs/BrainStation/Notebooks/proj_books/Capstone stuff/game_reviews'
        temp.to_csv(f'{path}/{i}_reviews.csv', index=False)


In [20]:
# batch 1 of reviews
get_data(app_id_0)

Current App: 353360

In [28]:
# batch 2 of reviews
get_data(app_id_1)

Current App: 501900

In [29]:
# batch 3 of reviews
get_data(app_id_2)

Current App: 651010

In [30]:
# batch 4 of reviews
get_data(app_id_3)

Current App: 790080

In [31]:
# final batch of reviews
get_data(app_id_4)

Current App: 1069460

In [35]:
# Creating initial dataframe on which all other csv data will be appended
path = 'C:/Users/Jonah Melhado/Documents/Important Docs/BrainStation/Notebooks/proj_books/Capstone stuff/game_reviews'

df_init = pd.read_csv(f'{path}/10_reviews.csv')

df_init.head()

,username,review,hours
0,Jormund,Recommended,"2,050.5 hrs on record"
1,Strikey,Recommended,"7,460.8 hrs on record"
2,꧁꧂,Recommended,"6,879.0 hrs on record"
3,Ace1,Recommended,193.6 hrs on record
4,Acc idle,Recommended,"1,931.8 hrs on record"


In [109]:
df_init.loc[df_init['review'] == 'Recommended', 'review'] = 1
df_init.loc[df_init['review'] != 1, 'review'] = 0

df_init.head() 

df_init_reviews = df_init[['username', 'review']]
df_init_reviews.rename(columns={'review': apps.loc[apps['appid']==10,'name'][0]}, inplace=True)

df_init_hours = df_init[['username', 'hours']]
df_init_hours.rename(columns={'hours': apps.loc[apps['appid']==10,'name'][0]}, inplace=True)

display(df_init_reviews.head())
display(df_init_hours.head())

C:\Users\TEMP\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,username,Counter-Strike
0,Jormund,1
1,Strikey,1
2,꧁꧂,1
3,Ace1,1
4,Acc idle,1


,username,Counter-Strike
0,Jormund,"2,050.5 hrs on record"
1,Strikey,"7,460.8 hrs on record"
2,꧁꧂,"6,879.0 hrs on record"
3,Ace1,193.6 hrs on record
4,Acc idle,"1,931.8 hrs on record"


In [47]:
# Creating initial dataframe that will become a dataframe of hours played of each game for each user
df_init_hours['Counter-Strike'] = df_init_hours['Counter-Strike'].str.replace(' hrs on record', '').str.replace(',', '').astype(float)
df_init_hours.head()

<ipython-input-47-d36c1abd8c2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_init_hours['Counter-Strike'] = df_init_hours['Counter-Strike'].str.replace(' hrs on record', '').str.replace(',', '').astype(float)


,username,Counter-Strike
0,Jormund,2050.5
1,Strikey,7460.8
2,꧁꧂,6879.0
3,Ace1,193.6
4,Acc idle,1931.8


In [97]:
df_test = df_test.fillna(-1).drop(columns=['hours'])

In [110]:
# Using csv's above to combine into one large review dataframe to then save as a large csv file
for i in app_id_0[1:1000]:
    
    # reading in temp dataframes for reviews
    df_temp_reviews = pd.read_csv(f'{path}/{i}_reviews.csv')
    df_temp_reviews.drop(columns=['hours'], inplace=True)
    
    # reformatting review column to 1 (Recommended) or 0 (Not Recommended)
    df_temp_reviews.loc[df_temp_reviews['review'] == 'Recommended', 'review'] = 1
    df_temp_reviews.loc[df_temp_reviews['review'] != 1, 'review'] = 0
    
    # renaming review column
    new_name = apps.iloc[apps.loc[apps['appid']==i, 'name'].index[0], 1]
    df_temp_reviews.rename(columns={'review': new_name}, inplace=True)
    
    # merging dataframes
    df_init_reviews = df_init_reviews.merge(df_temp_reviews, on='username', how='outer')
    df_init_reviews = df_init_reviews.fillna(-1)
    
    # keeping track of progress
    print(f'\r{df_init_reviews.shape}', end='')

(4981321, 333)

MemoryError: Unable to allocate 22.2 GiB for an array with shape (332, 8963402) and data type float64

In [122]:
# Checking final dataframe
df_init_reviews.head()

,username,Counter-Strike,Team Fortress Classic,Day of Defeat,Deathmatch Classic,Half-Life: Opposing Force,Ricochet,Half-Life,Counter-Strike: Condition Zero,Half-Life: Blue Shift,...,Rise of the Argonauts,Damnation,Overlord II,Operation Flashpoint: Dragon Rising,Ninja Reflex: Steamworks Edition,America's Army 3,Unreal 2: The Awakening,Unreal Tournament 3 Black,Unreal Tournament 2004: Editor's Choice Edition,Unreal Tournament: Game of the Year Edition
0,Jormund,1.0,-1.0,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
1,Strikey,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
2,꧁꧂,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
3,Ace1,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
4,Acc idle,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN
